In [1]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import wandb

wandb.init(project="mab-sst2")

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pd90506 (pd90506-nd). Use `wandb login --relogin` to force relogin


In [2]:
from llmexp.utils.data_utils import LLMDataset, create_dataloader
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."


In [3]:
from llmexp.explainer.mab_model import MABModel
from llmexp.trainer.mab_trainer import MABTrainer

mab_model = MABModel(llm, hidden_size=1024)

config = {
    'lr': 1e-4,
    'num_pulls': 3,
    'num_epochs': 1,
    'batch_size': 32,
    'clip_epsilon': 0.2,
    'minibatch_size': 32,
}
dataloader = create_dataloader('sst2', tokenizer, batch_size=config['batch_size'], instruction=instruction)

mab_trainer = MABTrainer(mab_model, llm, tokenizer, config, device)


In [4]:
mab_trainer.train(dataloader)


Training MAB: 0it [00:00, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
Training MAB: 540it [41:17,  4.50s/it]

In [5]:
# save the model
import torch
torch.save(mab_model.state_dict(), f"checkpoints/mab_model.pth")